In [1]:
import sys, os
import numpy as np

from evoman.environment import Environment
from evoman.controller import Controller

pygame 2.5.1 (SDL 2.28.2, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [14]:
model.get_weights()[0].shape

(20, 16)

In [2]:
def weights_matrix(model,solution):

    weights = solution[5:]
    bias = solution[:5]

    weigths_matrix = [weights.reshape(model.get_weights()[0].shape),bias.reshape(model.get_weights()[1].shape)]

    return weigths_matrix

def weights_matrix_to_vector(matrix):
    return np.concatenate([matrix[1],matrix[0].reshape(-1,)])
    

In [3]:
class player_controller(Controller):

	def __init__(self, model):
		self.model = model

	def control(self, inputs, controller):

		# Normalises the input using min-max scaling
		inputs = (inputs-min(inputs))/float((max(inputs)-min(inputs)))
		inputs = np.array(inputs).reshape((1,20,))

		sol = weights_matrix(self.model,controller)
		self.model.set_weights(sol)
		preds = self.model.predict(inputs,verbose=0)
		preds = (preds>0.5).astype(int)

		return preds[0]

In [4]:
headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

experiment_name = 'controller_specialist_demo'
if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

In [6]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

input_layer  = tf.keras.layers.Input((20,))
dense_layer1 = tf.keras.layers.Dense(16, activation="relu")(input_layer)
output_layer = tf.keras.layers.Dense(5, activation="sigmoid")(dense_layer1)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [7]:
env = Environment(experiment_name=experiment_name,
				  playermode="ai",
				  player_controller=player_controller(model),
			  	  speed="fastest",
				  enemymode="static",
                  #enemies="0",
				  level=2,
                  visuals=True)


MESSAGE: Pygame initialized for simulation.


In [8]:
sol = weights_matrix_to_vector(model.get_weights())

In [10]:
env.update_parameter('enemies',[1])


PARAMETER CHANGE: enemies = [1]


In [10]:
sol = np.loadtxt('solutions_demo/demo_'+str(1)+'.txt')

In [11]:
env.play(sol)

ValueError: cannot reshape array of size 331 into shape (20,16)